### Cuaderno ingesta de datos SECOP

Este cuaderno realiza un proceso de ingesta inteligente de datos del SECOP (Sistema Electrónico para la Contratación Pública) directamente desde el portal de datos abiertos de Colombia (`www.datos.gov.co`) utilizando la API de Socrata. Primero, se autentica el acceso con un token seguro y se selecciona dinámicamente el conjunto de datos deseado. Luego, se extraen los registros contractuales para cada entidad estatal (identificada por su NIT) y se almacenan en una tabla Delta (`secop_bronze`). El script realiza la descarga de forma controlada, aplicando reintentos ante errores y almacenando los resultados uno a uno. Además, incluye una segunda etapa que verifica cuáles NITs aún no han sido cargados previamente y solo descarga esos registros faltantes, optimizando así el proceso de actualización incremental y evitando duplicados. Este enfoque garantiza eficiencia, robustez y escalabilidad en la construcción de una capa de datos confiable para análisis posteriores.


In [0]:
from sodapy import Socrata

#client = Socrata("www.datos.gov.co", None)

token = dbutils.secrets.get("claves","token_app")
codigo_dataset = dbutils.widgets.get("codigo_dataset").strip()

client = Socrata("www.datos.gov.co",str(token),timeout=1000)

In [0]:
df_secop_id=spark.table("main.diplomado_datos.ids_contratos_procesos")
lista_entidades=df_secop_id.select("nit_de_la_entidad").distinct().collect()
len(list(map(lambda x: x.nit_de_la_entidad,lista_entidades)))

In [0]:
import time

mode_write = "overwrite"
reintentos = 5

for entidad in lista_entidades:
  intentos = 0
  while intentos < reintentos:
    try:
      query="""
      SELECT * WHERE nit_de_la_entidad = '{0}'
      """.format(entidad.nit_de_la_entidad)
      print(query)
      results_entidad=client.get(codigo_dataset,query=query)
      df_total=spark.createDataFrame(results_entidad)
      df_total.write \
        .format("delta") \
        .mode(mode_write) \
        .option("overwriteSchema", "true") \
        .saveAsTable("main.diplomado_datos.secop_bronze")
      mode_write = "append"
      break
    except Exception as e:
      intentos += 1
      time.sleep(10)
      print("Error: {0}".format(e))


print("Se cargaron todos los contratos")

In [0]:
# Cargar los NITs de la tabla original mínima
df_ids = spark.table("main.diplomado_datos.ids_contratos_procesos")
lista_entidades = df_ids.select("nit_de_la_entidad").distinct().collect()

# Cargar los NITs que ya se han cargado en bronze
df_bronze = spark.table("main.diplomado_datos.secop_bronze")
nits_cargados = [row["nit_de_la_entidad"] for row in df_bronze.select("nit_de_la_entidad").distinct().collect()]

# Filtrar los que aún no han sido cargados
nits_faltantes = [x.nit_de_la_entidad for x in lista_entidades if x.nit_de_la_entidad not in nits_cargados]

print(f"🔍 NITs faltantes por cargar: {len(nits_faltantes)}")

# ======================================
# Cargar solo los NITs faltantes (reintentos por cada uno)
# ======================================
from time import sleep

modo_escritura = "append"  # Ya no es overwrite
reintentos = 5

for nit in nits_faltantes:
    intentos = 0
    while intentos < reintentos:
        try:
            query = f"SELECT * WHERE nit_de_la_entidad = '{nit}'"
            print(query)
            resultados = client.get(codigo_dataset, query=query)
            df_temp = spark.createDataFrame(resultados)

            df_temp.write \
                .format("delta") \
                .mode(modo_escritura) \
                .option("overwriteSchema", "true") \
                .saveAsTable("main.diplomado_datos.secop_bronze")

            break  # Salta al siguiente NIT
        except Exception as e:
            intentos += 1
            print(f"Error al cargar NIT {nit} (intento {intentos}): {e}")
            sleep(10)

print("✅ Finalizó la carga de los registros faltantes.")